In [19]:
import pandas as pd 
import numpy as np

### 절대모멘텀
- 파일 로드 
- 파생변수 'STD-YM' 생성 -> Date 컬럼의 값을 년-월 추출 대입
- 년,월 별 마지막날 month_last_dt 데이터프레임 삽입 
- 전월의 종가의 값을 가지는 파생변수 생성
- 전년도의 종가의 값을 가지는 파생변수 생성
- 전월, 전년도의 종가를 가지고 거래내역 생성
- 수익율 계산

In [20]:
df = pd.read_csv("../csv/SPY.csv")
df.head(1)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-01-29,43.96875,43.96875,43.75,43.9375,26.706757,1003200


In [21]:
## 새로운 파생변수 STD-YM 생성
## Date 컬럼을 기준으로 년-월 추출해서 파생변수 대입

## Date 컬럼을 데이터형을 datetime 변경 ( pd.to_datetime() )
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
## STD-YM 컬럼을 만들어서 Date컬럼에서 년(4자리)-월 추출
df['STD-YM'] = df['Date'].dt.strftime("%Y-%m")

In [22]:
month_last_df = pd.DataFrame()

month_list = df['STD-YM'].unique()

for i in month_list:
    last_df = df[df['STD-YM'] == i].tail(1)
    month_last_df = pd.concat([month_last_df, last_df])
month_last_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 0 to 6647
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       318 non-null    datetime64[ns]
 1   Open       318 non-null    float64       
 2   High       318 non-null    float64       
 3   Low        318 non-null    float64       
 4   Close      318 non-null    float64       
 5   Adj Close  318 non-null    float64       
 6   Volume     318 non-null    int64         
 7   STD-YM     318 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 22.4+ KB


In [23]:
## 년-월 별 마지막날의 데이터들을 추출 
## 새로운 데이터프레임에 결합
month_last_df = df[~(df.shift(-1)['STD-YM'] == df['STD-YM'])]
month_last_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 0 to 6647
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       318 non-null    datetime64[ns]
 1   Open       318 non-null    float64       
 2   High       318 non-null    float64       
 3   Low        318 non-null    float64       
 4   Close      318 non-null    float64       
 5   Adj Close  318 non-null    float64       
 6   Volume     318 non-null    int64         
 7   STD-YM     318 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 22.4+ KB


In [24]:
## 'Date' 컬럼을 인덱스 변환
month_last_df.set_index(['Date'], inplace=True)

In [25]:
month_last_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,26.706757,1003200,1993-01
1993-02-26,44.43750,44.43750,44.18750,44.40625,26.991682,66200,1993-02
1993-03-31,45.34375,45.46875,45.18750,45.18750,27.596252,111600,1993-03
1993-04-30,44.12500,44.28125,44.03125,44.03125,26.890142,88500,1993-04
1993-05-28,45.40625,45.40625,45.00000,45.21875,27.615356,79100,1993-05
1993-06-30,45.12500,45.21875,45.00000,45.06250,27.714964,437600,1993-06
1993-07-30,45.09375,45.09375,44.78125,44.84375,27.580431,75300,1993-07
1993-08-31,46.40625,46.56250,46.34375,46.56250,28.637531,66500,1993-08
1993-09-30,46.03125,46.12500,45.84375,45.93750,28.429131,99300,1993-09


In [26]:
## month_last_df 파생변수 2개 생성
## 'BF_1M' 전월의 종가(Adj Close)의 값
## 'BF_12M' 전년도의 종가(Adj Close)의 값
## NaN이 나온다면 0으로 대체
month_last_df["BF_1M"] = month_last_df.shift(1)['Adj Close'].fillna(0)
month_last_df["BF_12M"] = month_last_df.shift(12)['Adj Close'].fillna(0)
month_last_df.iloc[10:15]

C:\Users\msi\AppData\Local\Temp\ipykernel_17084\711112444.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_last_df["BF_1M"] = month_last_df.shift(1)['Adj Close'].fillna(0)
C:\Users\msi\AppData\Local\Temp\ipykernel_17084\711112444.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_last_df["BF_12M"] = month_last_df.shift(12)['Adj Close'].fillna(0)


,Open,High,Low,Close,Adj Close,Volume,STD-YM,BF_1M,BF_12M
Date,,,,,,,,,
1993-11-30,46.28125,46.5625,46.25000,46.34375,28.680576,230000,1993-11,28.990005,0.000000
1993-12-31,46.93750,47.0000,46.56250,46.59375,29.032679,312900,1993-12,28.680576,0.000000
1994-01-31,48.06250,48.3125,48.00000,48.21875,30.045223,313800,1994-01,29.032679,26.706757
1994-02-28,46.93750,47.0625,46.81250,46.81250,29.168985,333000,1994-02,30.045223,26.991682
1994-03-31,44.46875,44.6875,43.53125,44.59375,27.946764,788800,1994-03,29.168985,27.596252


In [27]:
month_last_df.loc['1993-01-29':'1993-09-30']

,Open,High,Low,Close,Adj Close,Volume,STD-YM,BF_1M,BF_12M
Date,,,,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,26.706757,1003200,1993-01,0.000000,0.0
1993-02-26,44.43750,44.43750,44.18750,44.40625,26.991682,66200,1993-02,26.706757,0.0
1993-03-31,45.34375,45.46875,45.18750,45.18750,27.596252,111600,1993-03,26.991682,0.0
1993-04-30,44.12500,44.28125,44.03125,44.03125,26.890142,88500,1993-04,27.596252,0.0
1993-05-28,45.40625,45.40625,45.00000,45.21875,27.615356,79100,1993-05,26.890142,0.0
1993-06-30,45.12500,45.21875,45.00000,45.06250,27.714964,437600,1993-06,27.615356,0.0
1993-07-30,45.09375,45.09375,44.78125,44.84375,27.580431,75300,1993-07,27.714964,0.0
1993-08-31,46.40625,46.56250,46.34375,46.56250,28.637531,66500,1993-08,27.580431,0.0
1993-09-30,46.03125,46.12500,45.84375,45.93750,28.429131,99300,1993-09,28.637531,0.0


In [28]:
df.loc[1]

Date         1993-02-01 00:00:00
Open                    43.96875
High                       44.25
Low                     43.96875
Close                      44.25
Adj Close              26.896694
Volume                    480500
STD-YM                   1993-02
Name: 1, dtype: object

### loc, iloc
1. loc 
    - index와 column의 값을 비교하여 True인 행과 열을 출력
    - loc[index, columns]
    - loc[1] -> index의 값이 1인 행 출력
    - loc[:, column명]
2. iloc
    - index와 column의 위치를 출력
    - 데이터프레임에 2번째 행 출력 -> iloc[1]
    - 데이터프레림에서 2번째 열 출력 -> iloc[:,1]

In [29]:
df.set_index(['Date'], inplace=True)

In [30]:
## 거래내역 추가
## 구매 조건 -> 전월 종가 / 전년도 종가 - 1의 값이 0보다 크고 무한대가 아닌 경우
df['trade'] = ""

for i in month_last_df.index:
    signal = ""

    # 절대 모멘텀 계산
    momentum_index = month_last_df.loc[i, "BF_1M"] /\
         month_last_df.loc[i, "BF_12M"] - 1
    # print(momentum_index)
    # 절대 모멘텀 지표에 따라서 True / False 구분
    flag = True if((momentum_index > 0) and (momentum_index != np.inf) 
    and (momentum_index != -np.inf)) else False

    if flag:
        signal = 'buy'
    print("날짜 :", i, '모멘텀 인덱스 :', momentum_index, 
    "flag :", flag, 'signal :', signal)
    df.loc[i, "trade"] = signal
df['trade'].value_counts()
    


날짜 : 1993-01-29 00:00:00 모멘텀 인덱스 : nan flag : False signal : 
날짜 : 1993-02-26 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-03-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-04-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-05-28 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-06-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-07-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-08-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-09-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-10-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-11-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-12-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1994-01-31 00:00:00 모멘텀 인덱스 : 0.08709114326385659 flag : True signal : buy
날짜 : 1994-02-28 00:00:00 모멘텀 인덱스 : 0.1131289632117034 flag : True signal : buy
날짜 : 1994-03-31 00:00:00 모멘텀 인덱스 : 0.05699081889816049 flag : True signal : buy
날짜 : 1994-04-29 0

C:\Users\msi\AppData\Local\Temp\ipykernel_17084\4266041059.py:9: RuntimeWarning: invalid value encountered in double_scalars
  momentum_index = month_last_df.loc[i, "BF_1M"] /\
C:\Users\msi\AppData\Local\Temp\ipykernel_17084\4266041059.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  momentum_index = month_last_df.loc[i, "BF_1M"] /\


       6401
buy     247
Name: trade, dtype: int64

In [31]:
rtn = 1.0
df['return'] = 1
buy = 0
sell = 0
for i in df.index:
    ##구매한 날짜를 체크 : 현재 행의 trade = 'buy' 전 행의 trade = ""인 경우
    if df.loc[i, 'trade'] == 'buy' and df.shift(1).loc[i, 'trade'] == "":
        buy = df.loc[i, 'Adj Close']
        print("구매일 :", i, "구매 가격 :", buy)
    elif df.loc[i, 'trade'] == "" and df.shift(1).loc[i, 'trade'] == 'buy':
        sell = df.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1
        df.loc[i, 'return'] = rtn
        print("판매일 : ", i, "판매 가격 :", sell, "수익율 :", rtn)

    if df.loc[i, 'trade'] == "":
        buy = 0 
        sell = 0 


구매일 : 1994-01-31 00:00:00 구매 가격 : 30.045223
판매일 :  1994-02-01 00:00:00 판매 가격 : 29.889435 수익율 : 0.9948148828850429
구매일 : 1994-02-28 00:00:00 구매 가격 : 29.168985
판매일 :  1994-03-01 00:00:00 판매 가격 : 29.052134 수익율 : 0.995993998419897
구매일 : 1994-03-31 00:00:00 구매 가격 : 27.946764
판매일 :  1994-04-04 00:00:00 판매 가격 : 27.5159 수익율 : 0.9845826872835794
구매일 : 1994-04-29 00:00:00 구매 가격 : 28.260098
판매일 :  1994-05-02 00:00:00 판매 가격 : 28.436377 수익율 : 1.0062377349151443
구매일 : 1994-05-31 00:00:00 구매 가격 : 28.710556
판매일 :  1994-06-01 00:00:00 판매 가격 : 28.837847 수익율 : 1.00443359578268
구매일 : 1994-06-30 00:00:00 구매 가격 : 28.052668
판매일 :  1994-07-01 00:00:00 판매 가격 : 28.111818 수익율 : 1.0021085338478322
구매일 : 1994-07-29 00:00:00 구매 가격 : 28.959522
판매일 :  1994-08-01 00:00:00 판매 가격 : 29.097498 수익율 : 1.0047644432805212
구매일 : 1994-08-31 00:00:00 구매 가격 : 30.063478
판매일 :  1994-09-01 00:00:00 판매 가격 : 29.964916 수익율 : 0.9967215370091245
구매일 : 1994-09-30 00:00:00 구매 가격 : 29.30423
판매일 :  1994-10-03 00:00:00 판매 가격 : 29.234806 수익율 :

In [32]:
acc_rtn = 1 

for i in df.index:
    rtn = df.loc[i, 'return']
    acc_rtn *= rtn
    df.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익율 :', acc_rtn)

누적 수익율 : 2.0162118510060383


### 절대 모멘텀 함수로 구현
1. 1번 함수
    - 매개변수 2개 ( 데이터프레임, 기준이 되는 컬럼 )
    - 데이터프레임에 결측치와 이상치 제거
    - Date, 기준이 되는 컬럼 두 컬럼만 두고 나머지 컬럼은 삭제
    - Date 컬럼의 데이터를 시계열데이터 변경
    - 'STD-YM' 파생변수를 생성을 하여 년-월 대입
    - 인덱스를 Date로 변경
    - 수정이 된 데이터프레임을 리턴
    - 1번 함수 테스트하실때는 csv 파일 새로 불러와서 함수호출
2. 2번 함수
    - 매개변수(df) 1개( 데이터프레임 )
    - 새로운 데이터프레임 생성 
    - df의 데이터에서 년-월별 마지막 데이터들을 새로운 데이터프레임 삽입
    - 새로운 데이터프레임에 파생변수 2개 생성
    - 'BF_1M' : 전 월의 종가, 결측치는 0으로 대체
    - 'BF_12M' : 전 년의 종가, 결측치는 0으로 대체
    - 새로운 데이터프레임을 리턴


In [35]:
'Close' in df1.columns

NameError: name 'df1' is not defined

In [36]:
## 1번 함수 생성
def first(data, col):
    ##조건식을 이용하여 데이터형태를 같게 csv / yfinance
    # if type(data.index[0]) != int:
    if 'Date' not in data.columns:
        # data = data.reset_index()
        data.reset_index(inplace=True)

    ## 결측치와 이상치를 제거
    result = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]
    result = result.loc[:, ['Date', col]] ## result[['Date', col]]
    result['Date'] = pd.to_datetime(result['Date'])
    result['STD-YM'] = result['Date'].dt.strftime("%Y-%m")
    result.set_index('Date', inplace=True)
    return result

In [37]:
test_df = pd.read_csv("../csv/AAPL.csv")

test_df = first(test_df, 'Close')

In [38]:
## 2번 함수
def second(data):
    result = pd.DataFrame()
    col = data.columns[0]
    data = data[~(data.shift(-1)['STD-YM'] == data['STD-YM'])]
    result = pd.concat([result, data])
    result['BF_1M'] = result.shift(1)[col].fillna(0)
    result['BF_12M'] = result.shift(12)[col].fillna(0)
    return result
    

In [39]:
test_df_2 = second(test_df)

3. 3번 함수 생성
    - 매개변수 2개 (데이터프레임(df1), 데이터프레임(df2))
    - df1에 trade 컬럼 생성 값은 ""
    - df1에 return 컬럼 생성 값은 1
    - df2의 값들을 이용하여 momentum index를 구하고 df1에 거래 내역 대입
    - df1의 거래내역을 이용하여 수익율 return 대입
    - return의 값들은 가지고 누적 수익율을 acc_rtn 컬럼에 대입
    - 누적 수익율을 print()
    - df1을 리턴

In [40]:
## 3번 함수
def third(df1, df2):
    df1["trade"] = ""
    df1['return'] = 1
    for i in df2.index:
        signal = ""

        # 절대 모멘텀 계산
        momentum_index = df2.loc[i, "BF_1M"] /\
            df2.loc[i, "BF_12M"] - 1
        # print(momentum_index)
        # 절대 모멘텀 지표에 따라서 True / False 구분
        flag = True if((momentum_index > 0) and (momentum_index != np.inf) 
        and (momentum_index != -np.inf)) else False

        if flag:
            signal = 'buy'
        print("날짜 :", i, '모멘텀 인덱스 :', momentum_index, 
        "flag :", flag, 'signal :', signal)
        df1.loc[i, "trade"] = signal

    rtn = 1.0
    buy = 0
    sell = 0
    col = df1.columns[0]
    for i in df1.index:
        ##구매한 날짜를 체크 : 현재 행의 trade = 'buy' 전 행의 trade = ""인 경우
        if df1.loc[i, 'trade'] == 'buy' and df1.shift(1).loc[i, 'trade'] == "":
            buy = df1.loc[i, col]
            print("구매일 :", i, "구매 가격 :", buy)
        elif df1.loc[i, 'trade'] == "" and df1.shift(1).loc[i, 'trade'] == 'buy':
            sell = df1.loc[i, col]
            rtn = (sell - buy) / buy + 1
            df1.loc[i, 'return'] = rtn
            print("판매일 : ", i, "판매 가격 :", sell, "수익율 :", rtn)

        if df1.loc[i, 'trade'] == "":
            buy = 0 
            sell = 0 
    
    acc_rtn = 1 

    for i in df1.index:
        rtn = df1.loc[i, 'return']
        acc_rtn *= rtn
        df1.loc[i, 'acc_rtn'] = acc_rtn

    print('누적 수익율 :', acc_rtn)

    return df1
    
    
    

In [41]:
third(test_df, test_df_2)

C:\Users\msi\AppData\Local\Temp\ipykernel_17084\1521845317.py:9: RuntimeWarning: invalid value encountered in double_scalars
  momentum_index = df2.loc[i, "BF_1M"] /\
C:\Users\msi\AppData\Local\Temp\ipykernel_17084\1521845317.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  momentum_index = df2.loc[i, "BF_1M"] /\


날짜 : 1980-12-31 00:00:00 모멘텀 인덱스 : nan flag : False signal : 
날짜 : 1981-01-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-02-27 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-03-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-04-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-05-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-06-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-07-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-08-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-09-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-10-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-11-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1981-12-31 00:00:00 모멘텀 인덱스 : -0.45421292307692307 flag : False signal : 
날짜 : 1982-01-29 00:00:00 모멘텀 인덱스 : -0.21681428208950493 flag : False signal : 
날짜 : 1982-02-26 00:00:00 모멘텀 인덱스 : -0.23113221502322412 flag : False signal : 
날짜 : 1982-03-31 00:

,Close,STD-YM,trade,return,acc_rtn
Date,,,,,
1980-12-12,0.513393,1980-12,,1.0,1.000000
1980-12-15,0.486607,1980-12,,1.0,1.000000
1980-12-16,0.450893,1980-12,,1.0,1.000000
1980-12-17,0.462054,1980-12,,1.0,1.000000
1980-12-18,0.475446,1980-12,,1.0,1.000000
...,...,...,...,...,...
2019-06-18,198.449997,2019-06,,1.0,2.227569
2019-06-19,197.869995,2019-06,,1.0,2.227569
2019-06-20,199.460007,2019-06,,1.0,2.227569


In [42]:
import yfinance as yf

In [43]:
yf_df = yf.download('003490.KS', start='2018-01-01')

[*********************100%***********************]  1 of 1 completed


In [44]:
df1 = first(yf_df, 'Close')
df1.head()

,Close,STD-YM
Date,,
2018-01-02,35900.0,2018-01
2018-01-03,35750.0,2018-01
2018-01-04,34900.0,2018-01
2018-01-05,34900.0,2018-01
2018-01-08,36100.0,2018-01


In [45]:
import momentum as mmt

In [46]:
mmt1 = mmt.Momentum(yf_df, 'Adj Close')

In [47]:
mmt1.testing()

c:\ubion\221222\momentum\__init__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df2['BF_1M'] = self.df2.shift(1)[self.col].fillna(0)
c:\ubion\221222\momentum\__init__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df2['BF_12M'] = self.df2.shift(12)[self.col].fillna(0)
c:\ubion\221222\momentum\__init__.py:34: RuntimeWarning: invalid value encountered in double_scalars
  momentum_index = self.df2.loc[i, "BF_1M"] /\
c:\ubion\221222\momentum\__init__.py:34: RuntimeWarning: divide by zero 

날짜 : 2018-01-31 00:00:00 모멘텀 인덱스 : nan flag : False signal : 
날짜 : 2018-02-28 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-03-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-04-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-05-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-06-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-07-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-08-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-09-28 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-10-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-11-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2018-12-28 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2019-01-31 00:00:00 모멘텀 인덱스 : -0.13823501551743278 flag : False signal : 
날짜 : 2019-02-28 00:00:00 모멘텀 인덱스 : 0.12062415532882143 flag : True signal : buy
날짜 : 2019-03-29 00:00:00 모멘텀 인덱스 : 0.09439200766706368 flag : True signal : buy
날짜 : 2019-04-30 0

,Adj Close,STD-YM,trade,return,acc_rtn
Date,,,,,
2018-01-02,35715.164062,2018-01,,1.0,1.000000
2018-01-03,35565.937500,2018-01,,1.0,1.000000
2018-01-04,34720.312500,2018-01,,1.0,1.000000
2018-01-05,34720.312500,2018-01,,1.0,1.000000
2018-01-08,35914.136719,2018-01,,1.0,1.000000
...,...,...,...,...,...
2022-12-16,24700.000000,2022-12,,1.0,1.072949
2022-12-19,24600.000000,2022-12,,1.0,1.072949
2022-12-20,24450.000000,2022-12,,1.0,1.072949


In [48]:
t_df = pd.read_csv("../csv/AMZN.csv")

In [49]:
mmt2 = mmt.Momentum(t_df, 'Adj Close')

In [50]:
mmt2.testing()

c:\ubion\221222\momentum\__init__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df2['BF_1M'] = self.df2.shift(1)[self.col].fillna(0)
c:\ubion\221222\momentum\__init__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df2['BF_12M'] = self.df2.shift(12)[self.col].fillna(0)
c:\ubion\221222\momentum\__init__.py:34: RuntimeWarning: invalid value encountered in double_scalars
  momentum_index = self.df2.loc[i, "BF_1M"] /\
c:\ubion\221222\momentum\__init__.py:34: RuntimeWarning: divide by zero 

날짜 : 1997-05-30 00:00:00 모멘텀 인덱스 : nan flag : False signal : 
날짜 : 1997-06-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1997-07-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1997-08-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1997-09-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1997-10-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1997-11-28 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1997-12-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1998-01-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1998-02-27 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1998-03-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1998-04-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1998-05-29 00:00:00 모멘텀 인덱스 : 4.0972219999999995 flag : True signal : buy
날짜 : 1998-06-30 00:00:00 모멘텀 인덱스 : 3.7635124835648686 flag : True signal : buy
날짜 : 1998-07-31 00:00:00 모멘텀 인덱스 : 5.9391314002269775 flag : True signal : buy
날짜 : 1998-08-31 00:

,Adj Close,STD-YM,trade,return,acc_rtn
Date,,,,,
1997-05-15,1.958333,1997-05,,1.0,1.00000
1997-05-16,1.729167,1997-05,,1.0,1.00000
1997-05-19,1.708333,1997-05,,1.0,1.00000
1997-05-20,1.635417,1997-05,,1.0,1.00000
1997-05-21,1.427083,1997-05,,1.0,1.00000
...,...,...,...,...,...
2019-06-18,1901.369995,2019-06,,1.0,1.03362
2019-06-19,1908.790039,2019-06,,1.0,1.03362
2019-06-20,1918.189941,2019-06,,1.0,1.03362
